# Advanced: `DAGConfigurator`

Out of the box, `DAG` has just a few customizable parameters. `DAGConfigurator` provides more way to customize DAGs, the rationale for creating a new object instead of just adding more parameters to the DAG constructor is to keep the API simple for new users who might not even need to use `DAGConfigurator` at all.

The following parameters are available:

1. `outdated_by_code` (`bool`): By default, source code changes mark tasks as outdated, triggering execution during `dag.build()`, by setting this parameter to False, changes in code will not trigger task execution. This is useful to prevents in production override previous results if a new version (with new source) is deployed
2. `cache_rendered_status` (`bool`): By default, every time you run `dag.render()` or `dag.build()`, every Task fetches their corresponding Product status. Sometimes this is an expensive operation, for example, if Products are SQL tables, checking Product status has to query the database, by setting this to False, the status is cached.
3. `differ` (`CodeDiffer` object): To determine if two pieces of code are different, DAGs use a `CodeDiffer` that compare the old and the new version, by passing a custom object here, you can override the default behavior, see `CodeDiffer` documentation for details.
4. `logging_handler_factory` (`function`): This parameter provides an easy way to log `dag.build()` output. Whenever a DAG is built, this function is executed to add a logging handler to the root logger, after executing the DAG, the handler is removed. For more information about handlers and loggers, refer to the [logging module documentation](https://docs.python.org/3/howto/logging.html#logging-basic-tutorial). This function can request the DAG's name using the `dag_name` parameter.

## Logging all dag builds to a file

We now explain how to use `DAGConfigurator` with an example that logs DAG builds to a file:

In [1]:
from pathlib import Path
import logging

from ploomber import DAGConfigurator
from ploomber.tasks import PythonCallable
from ploomber.products import File

def logging_handler_factory(dag_name):
    handler = logging.FileHandler('%s.log' % dag_name)
    handler.setLevel(logging.INFO)
    fmt = logging.Formatter('%(levelname)s %(asctime)s: %(message)s')
    handler.setFormatter(fmt)
    return handler

# initialize and set logging_handler_factory
configurator = DAGConfigurator()
configurator.param.logging_handler_factory = logging_handler_factory

In [2]:
# initialize an empty dag with the parameter in configurator
dag = configurator.create(name='my_dag')

In [3]:
# add sample task
def touch(product):
    logging.info('Logging from touch task...')
    Path(str(product)).touch()

PythonCallable(touch, File('some_file.txt'), dag=dag)

PythonCallable: touch -> File(some_file.txt)

In [4]:
dag.build()

INFO:root:Logging from touch task...
INFO:ploomber.tasks.Task.PythonCallable:Done. Operation took 0.1 seconds


name,Ran?,Elapsed (s),Percentage
touch,True,0.058744,100


Let's now verify that our output was logged:

In [5]:
print(Path('my_dag.log').read_text())

INFO 2020-05-03 15:51:20,819: Rendering DAG DAG("my_dag")
INFO 2020-05-03 15:51:20,847: Checking status for task "touch"
INFO 2020-05-03 15:51:20,847: Product does not exist...
INFO 2020-05-03 15:51:20,847: Should run? True
INFO 2020-05-03 15:51:20,852: Building DAG DAG("my_dag")
INFO 2020-05-03 15:51:20,901: Starting execution: PythonCallable: touch -> File(some_file.txt)
INFO 2020-05-03 15:51:20,903: Logging from touch task...
INFO 2020-05-03 15:51:20,905: Done. Operation took 0.0 seconds
INFO 2020-05-03 15:51:21,012:  DAG report:
name    Ran?      Elapsed (s)    Percentage
------  ------  -------------  ------------
touch   True         0.002176           100



## Initializing `DAGConfigurator` using a dictionary

You can also initialize `DAGConfigurator` using a dictionary (e.g. if you want to load parameters from a JSON or a YAML file).
This:

In [6]:
configurator = DAGConfigurator()
configurator.param.cache_rendered_status = True

Is equivalent to this:

In [7]:
configurator = DAGConfigurator({'cache_rendered_status': False})